# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [3]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) > P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) \approx P(комиксы | я, часто ,читаю) $$
Попробуйте объяснить выбор для каждого из примеров.

Ответ :

1) "мама мыла раму" - это устойчивое выражение, а "мама мыла папу" - не кажется очень частотным выражением, потому интуитивно вероятность первого будет выше.

2) для второго неравенства, аналогичные рассуждения, так как там сравниваются вероятноси для устойчивого выражения и для обычной фразы, которая даже звучит не очень логично для обычной речи.

3) Ваня - это более частотное имя, поэтому вероятность фразы "меня зовут Ваня" интуитивно будет больше вероятности фразы "меня зовут Евпатий"

4) в последнем равенстве сложно определить, для меня обе фразы кажутся одинаково частотными, поэтому ставлю примерно равно

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой ответ верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [4]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX' -O small_corp_for_test.txt

--2021-12-20 23:29:52--  https://docs.google.com/uc?export=download&id=1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX
Resolving docs.google.com (docs.google.com)... 172.217.193.138, 172.217.193.100, 172.217.193.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.193.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dpjriqga6crjgvdk9i9hkc9hf3pl94jc/1640042925000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX?e=download [following]
--2021-12-20 23:29:53--  https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dpjriqga6crjgvdk9i9hkc9hf3pl94jc/1640042925000/09709797766287458306/*/1ijpKzdj4d1b0zYKot3_380OWtiI-2UJX?e=download
Resolving doc-0s-54-docs.googleusercontent.com (doc-0s-54-docs.googleusercontent.com)... 108.177.11.132, 2607:f8b0:400c:c01::84
Connecting to doc-0s-54-docs.googleusercontent.com (doc-0s-

In [5]:
path = 'small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
len(data)

700000

Посмотрю на 10 случайных элементов файла:

In [6]:
ind = np.random.randint(low=0, high=len(data), size=10)
for i in ind:
    print(data[i])

ну на прописку буду подавать не я я собираю пакет документов

сейчас одну секундочку

я предприниматель да для меня самое важное

аа икс идея от профи это фоторепортаж

стражники хью с треском проламывались сквозь кусты и цепью смыкались внизу чтобы взять беглецов в кольцо

аа вы сказали что уже не работаете если

через сто двенадцать

алло да здравствуйте дмитрий это сергей слышу вас теперь

вот и конкуренты

кры крыши чтоб тебя



In [7]:
len(np.unique(data))

409380

Получается, что в обучающей выборке есть повторяющиеся тексты

In [8]:
unique_symbols = set()
for text in data:
    unique_symbols = unique_symbols.union(set(text))
len(unique_symbols)

36

In [9]:
unique_symbols

{'\n',
 ' ',
 '-',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё'}

Таким образом, во всех текстах содержатся только символы алфавита, тире, пробела и переноса строки.

Уберу из всех тексттов символы \n и \t, как просят в задании:

In [10]:
for i in range(len(data)):
    data[i] = data[i].replace('\n', '').replace('\t', '')

## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [11]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '    # не хватало буквы ф
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        text_preproc = text.lower()
        pad_size = window_size - len(text_preproc)
        text_preproc += pad_size * '_'

        text_vectorized = []
        for sym in text_preproc:
            text_vectorized.append(self.token2ind[sym])
        
        return [np.array(text_vectorized[:-1]), np.array(text_vectorized[1:])]

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [12]:
for i in range(len(data)):
    data[i] = '[' + data[i] + ']'

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [13]:
THRESHOLD = 128

mask = (np.array(list(map(len, data))) <= THRESHOLD)
sub_data = np.array(data)[mask]
mask.sum()

683438

То есть с данным значением порога получаюм выборку всего из 683438 текстов.

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
data_train, data_test, _, _ = train_test_split(sub_data, sub_data, test_size=0.15, shuffle=True, random_state=123)

## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [16]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = 128):
        self.preprocessor = preproc
        data = np.array(list(map(lambda text: preproc.preprocess(text, win_size), x)))
        self.x = torch.from_numpy(data[:, 0])
        self.y = torch.from_numpy(data[:, 1])
        self.window_size = win_size


    def __len__(self):
        return len(self.x)
    

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [17]:
preproc = Preprocessor()
train_dataset = TextDataset(data_train, preproc)
test_dataset = TextDataset(data_test, preproc)

In [18]:
train_dataset[0]

(tensor([36, 11, 37, 23,  8, 17, 37,  9, 20, 14, 37, 12,  8, 17, 37,  3,  9,  1,
         14, 12, 37, 10,  2, 10,  5,  2, 32, 14, 37, 23, 13, 14,  1,  5, 19, 12,
         27, 37, 10,  2, 12,  2, 17,  9, 37, 16, 12,  2, 37, 23, 25, 37, 10,  2,
         22, 17, 33, 12, 14, 37, 16, 12,  2, 37,  6,  8,  6, 11, 14, 28, 12,  2,
         37,  2, 29, 11,  3,  6, 11, 37,  3,  2, 21, 14, 14, 37, 23,  8, 20, 13,
         25, 37,  6,  8,  6, 11, 14, 28, 12,  2, 35,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0]),
 tensor([11, 37, 23,  8, 17, 37,  9, 20, 14, 37, 12,  8, 17, 37,  3,  9,  1, 14,
         12, 37, 10,  2, 10,  5,  2, 32, 14, 37, 23, 13, 14,  1,  5, 19, 12, 27,
         37, 10,  2, 12,  2, 17,  9, 37, 16, 12,  2, 37, 23, 25, 37, 10,  2, 22,
         17, 33, 12, 14, 37, 16, 12,  2, 37,  6,  8,  6, 11, 14, 28, 12,  2, 37,
          2, 29, 11,  3,  6, 11, 37,  3,  2, 21, 14, 14, 37, 23,  8, 20, 13, 25,
         37, 

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

Данный приём необходим для корректной работы механизма внимания. Смысл маскирования символов в том, чтобы не дать модели видеть, что будет в будующем. Для этого у матрицы скалярных произведений мы применяем маску, то есть отключаем элементы выше главной диагоняли. Благодаря этому, выход зависит не от всей последовательности целиком, а только он n-1 предыдущих символов при работе с символом под номером n.

In [19]:
import math

In [20]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [21]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        self.hidden_layer = 512
        self.emb = nn.Embedding(vocab_size, self.hidden_layer)
        self.pe = PositionalEncoding(self.hidden_layer)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(self.hidden_layer, nhead=1)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=6)
        self.lin_layer_size = 1024
        self.decoder = nn.Sequential(nn.Linear(self.hidden_layer, self.lin_layer_size),
                                     nn.ReLU(),
                                     nn.Linear(self.lin_layer_size, vocab_size))
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x))
        x = x.transpose(1, 0)
        x = self.transformer_encoder_layer(x, src_mask)
        x = self.decoder(x)
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [22]:
model = LanguageModel(len('_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '))

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [23]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                                            batch_size = self.train_batch_size,
                                                            shuffle = True,
                                                            pin_memory = True)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                                            batch_size = self.test_batch_size,
                                                            shuffle = True,
                                                            pin_memory = True)
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)
        
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index = 0)
                             # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                             # ignore index индекс символа _, чтобы модель не штрафовалась за то
                             # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            x = x.to(self.device)
            y = y.to(self.device)
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            scr_mask = self.model.generate_square_subsequent_mask(THRESHOLD - 1).to(self.device)
            predicted = self.model(x, scr_mask)
            loss = self.criterion(predicted.transpose(1, 2), y)
            counted_loss += loss
            step += 1
            it += 1
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            x = x.to(self.device)
            y = y.to(self.device)
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            scr_mask = self.model.generate_square_subsequent_mask(THRESHOLD - 1).to(self.device)
            predicted = self.model(x, scr_mask)
            loss = self.criterion(predicted.transpose(1, 2), y)
            counted_loss += loss
            step += 1
            it += 1
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        self.model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            self.model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                self.model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [24]:
trainer = Trainer(model, train_dataset, test_dataset)
trainer.train(5)

Train epoch 1 | Step 1000/9077 | Counted loss tensor(2501.7603, device='cuda:0', grad_fn=<AddBackward0>) | ppl 12.20395992132023 | time 36.480594873428345 | 
Train epoch 1 | Step 2000/9077 | Counted loss tensor(2403.5393, device='cuda:0', grad_fn=<AddBackward0>) | ppl 11.062261150372649 | time 35.63796281814575 | 
Train epoch 1 | Step 3000/9077 | Counted loss tensor(2356.8708, device='cuda:0', grad_fn=<AddBackward0>) | ppl 10.557862992492023 | time 35.75729966163635 | 
Train epoch 1 | Step 4000/9077 | Counted loss tensor(2319.9475, device='cuda:0', grad_fn=<AddBackward0>) | ppl 10.175142331391571 | time 35.70889067649841 | 
Train epoch 1 | Step 5000/9077 | Counted loss tensor(2294.1814, device='cuda:0', grad_fn=<AddBackward0>) | ppl 9.916317030881677 | time 35.89362668991089 | 
Train epoch 1 | Step 6000/9077 | Counted loss tensor(2279.5198, device='cuda:0', grad_fn=<AddBackward0>) | ppl 9.771986739170137 | time 36.01641869544983 | 
Train epoch 1 | Step 7000/9077 | Counted loss tensor(2

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [25]:
def generate_text(text):
    x = []
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x)).unsqueeze(0).to('cuda:0')
    
    pred = model(x, model.generate_square_subsequent_mask(len(text)).to('cuda:0'))
    ind = torch.argmax(pred[0][-1]).item()
    
    text += preproc.ind2token[ind] 
    
    if (len(text) > 150) or (text[-1] == ']'):
        return text
    else:
        return generate_text(text)

In [26]:
generate_text('[слово')

'[слововот вот т т вот то вот тако вот вот вот вот вот вот такот вот вот вот вот вот вот т ворот такот вот вот вот вот вот вот вот т такорот вот вот вот'

In [27]:
generate_text('[начало')

'[начало вено про посто посто по подато пому по поду пода пода пововоду поду поду поду поду поду поду поду пода пода пода пода пода пода пода пода поска'

In [28]:
generate_text('[полина')

'[полина пость получать по по податому по податому по по податому по по податому потому что что во нисто нисто ниму ниму на на по по по по по пому по по'

In [29]:
generate_text('[учёба')

'[учёба по пододоват ват поду поду пододолнить вать по по поду поду пода пода пода пода подоска пода подоска подоска повововоду поду подуму подуметотото'

In [30]:
generate_text('[выхухоль')

'[выхухоль сте подить поде подить подать пода пода пода пода пода пода пода пода пода пода пода пода пода полуска полуска пода пожа пожа пожа пожа пожа '

Получилось, что модель очень любит слоги 'по', но, тем не менее, иногда выдаёт что-то похожее на слова.

Я подбирала различные гиперпараметры, такие как ширина скрытых слоёв, количество скрытых слоёв и темп обучения. При очень широких скрытых слоях, модели нужно было много эпох для обучения и это занимало очень много времени, при маленькой ширине получалась очень плохая модель, она генерировала только пробелы или только один слог 'по' много раз подряд. Поэтому пришлось подобрать оптимальную ширину, такую что, модель уже научилась генерировать хоть какие-то слова и при этом училасть всего чуть меньше двух часов. Для этой задачи подходит очень небольшой тесп обучения, поэтому я выбрала 0.0001, хотя 0.00001 и меньше могут давать лучшее качество, но тогда модель очень долго учится. Было бы неплохо подобрать шедулер, чтобы он уменьшал тесм обучения, но на это мне, к сожалению, не хватило времени, уже время поджимало.

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [ ]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = ...
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        ################

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = ...
        self.head = ...
    
    def forward(self, x):
        x = ... # emb
        x = ... # head
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = ...
        self.test_dataloader = ...
        self.train_dataloader_size = ...
        self.test_dataloader_size = ...
        
        self.device = 'cuda:0'
        self.criterion = ... 
        
        self.optimizer = ...
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [ ]:
# YOUR CODE HERE
###############